In [1]:
import os
import pandas as pd
from glob import glob


dataset_path = '/home/uj-user/Yo/HiT5/HCLT/gen_dataset_curation/results/*'
dataset_path_list = glob(dataset_path)

save_path = '/home/uj-user/Yo/HiT5/HCLT/KoAlpaca/train_v1.1b/train_json/'
# df = pd.read_json('select_ver_5_10_rand_0.05.json', orient='records') 

os.makedirs(save_path, exist_ok=True)
for path in dataset_path_list:
    df = pd.read_json(path, orient='records', lines=True)
    df['text'] = '### 질문: ' + df['instruction'] + '### 답변: ' + df['output']
    preprocessed_df = df.loc[:, ['text']]
    preprocessed_df.to_json(save_path + path.split('/')[-1], orient='records', lines=True, force_ascii=False)

# Load State Dict

In [ ]:
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer
import tensor_parallel as tp

model = AutoModelForCausalLM.from_pretrained('/home/uj-user/Yo/HiT5/HCLT/KoAlpaca/train_v1.1b/polyglot-5.8b-result_len5_k10_mppl_h_n0.1.json-A30-e20')
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/polyglot-ko-5.8b')
tp_model = tp.tensor_parallel(model)

[2023-09-06 01:08:28,213] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at /home/uj-user/Yo/HiT5/HCLT/KoAlpaca/train_v1.1b/polyglot-5.8b-result_len5_k10_mppl_h_n0.1.json-A30-e20 were not used when initializing GPTNeoXForCausalLM: ['wrapped_model.module_shards.0.gpt_neox.layers.11.mlp.tp_wrapped_module.dense_h_to_4h.bias', 'wrapped_model.module_shards.3.gpt_neox.layers.23.post_attention_layernorm.weight', 'wrapped_model.module_shards.0.gpt_neox.layers.20.attention.tp_wrapped_module.dense.bias', 'wrapped_model.module_shards.3.gpt_neox.layers.19.mlp.tp_wrapped_module.dense_4h_to_h.bias', 'wrapped_model.module_shards.2.gpt_neox.layers.15.attention.tp_wrapped_module.rotary_emb.inv_freq', 'wrapped_model.module_shards.1.gpt_neox.layers.18.attention.tp_wrapped_module.bias', 'wrapped_model.module_shards.0.gpt_neox.layers.1.post_attention_layernorm.weight', 'wrapped_model.module_shards.1.gpt_neox.layers.14.attention.tp_wrapped_module.dense.weight', 'wrapped_model.module_shards.1.gpt_neox.layers.4.attention.tp_wrapped_module.rotar

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=tp_model,
    tokenizer=tokenizer
)
# trainer.save_model('./tp_model')
trainer.save_state()

In [ ]:
tp_model_state_dict = tp_model.state_dict()
tp_model_state_dict

OrderedDict([('wrapped_model.module_shards.0.gpt_neox.embed_in.tp_wrapped_module.weight',
              tensor([[-0.0094, -0.0036,  0.0189,  ..., -0.0075, -0.0315, -0.0150],
                      [ 0.0242,  0.0158,  0.0185,  ..., -0.0243, -0.0069,  0.0163],
                      [-0.0384, -0.0017, -0.0297,  ...,  0.0190, -0.0134, -0.0178],
                      ...,
                      [-0.0143, -0.0287,  0.0050,  ..., -0.0162, -0.0294,  0.0152],
                      [ 0.0080,  0.0195, -0.0163,  ..., -0.0301,  0.0236,  0.0308],
                      [ 0.0022, -0.0152, -0.0037,  ...,  0.0194, -0.0316, -0.0178]],
                     device='cuda:0')),
             ('wrapped_model.module_shards.0.gpt_neox.layers.0.input_layernorm.weight',
              tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0')),
             ('wrapped_model.module_shards.0.gpt_neox.layers.0.input_layernorm.bias',
              tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')),
             ('wrapp

In [ ]:
tp_model_state_dict = tp_model.state_dict()
tp_model_state_dict

OrderedDict([('wrapped_model.module_shards.0.gpt_neox.embed_in.tp_wrapped_module.weight',
              tensor([[ 0.0011, -0.0008,  0.0022,  ...,  0.0003, -0.0022, -0.0020],
                      [-0.0005,  0.0003,  0.0011,  ..., -0.0013, -0.0017,  0.0001],
                      [-0.0013,  0.0023,  0.0054,  ...,  0.0052, -0.0041, -0.0026],
                      ...,
                      [ 0.0009,  0.0005, -0.0004,  ..., -0.0010,  0.0018, -0.0007],
                      [-0.0011, -0.0002,  0.0004,  ..., -0.0021,  0.0006,  0.0009],
                      [ 0.0009, -0.0008,  0.0008,  ...,  0.0028, -0.0004,  0.0002]],
                     device='cuda:0')),
             ('wrapped_model.module_shards.0.gpt_neox.layers.0.input_layernorm.weight',
              tensor([0.3037, 0.3167, 0.3555,  ..., 0.1136, 0.4805, 0.3401], device='cuda:0')),
             ('wrapped_model.module_shards.0.gpt_neox.layers.0.input_layernorm.bias',
              tensor([ 0.0428, -0.0123, -0.0808,  ...,  0.2588,  0.0

In [ ]:
model_dir = "/home/uj-user/Yo/HiT5/HCLT/KoAlpaca/train_v1.1b/polyglot-5.8b-result_len5_k10_mppl_h_n0.1.json-A30-e20"
model_filename_template = "pytorch_model-{:05d}-of-{:05d}.bin"

tp_model_state_dict = tp_model.state_dict()

tp_model_state_dict.update(torch.load(f"{model_dir}/{model_filename_template.format(1, 3)}"))
tp_model_state_dict.update(torch.load(f"{model_dir}/{model_filename_template.format(2, 3)}"))
tp_model_state_dict.update(torch.load(f"{model_dir}/{model_filename_template.format(3, 3)}"))

In [ ]:
tp_model_state_dict

OrderedDict([('wrapped_model.module_shards.0.gpt_neox.embed_in.tp_wrapped_module.weight',
              tensor([[ 0.0011, -0.0008,  0.0022,  ...,  0.0003, -0.0022, -0.0020],
                      [-0.0005,  0.0003,  0.0011,  ..., -0.0013, -0.0017,  0.0001],
                      [-0.0013,  0.0023,  0.0054,  ...,  0.0052, -0.0041, -0.0026],
                      ...,
                      [ 0.0009,  0.0005, -0.0004,  ..., -0.0010,  0.0018, -0.0007],
                      [-0.0011, -0.0002,  0.0004,  ..., -0.0021,  0.0006,  0.0009],
                      [ 0.0009, -0.0008,  0.0008,  ...,  0.0028, -0.0004,  0.0002]],
                     device='cuda:0')),
             ('wrapped_model.module_shards.0.gpt_neox.layers.0.input_layernorm.weight',
              tensor([0.3037, 0.3166, 0.3554,  ..., 0.1136, 0.4805, 0.3401], device='cuda:0')),
             ('wrapped_model.module_shards.0.gpt_neox.layers.0.input_layernorm.bias',
              tensor([ 0.0428, -0.0123, -0.0808,  ...,  0.2588,  0.0

### 안될때 

In [ ]:
output_model_file = "./models/my_own_model_file.bin"
output_config_file = "./models/my_own_config_file.bin"
output_vocab_file = "./models/my_own_vocab_file.bin"

In [ ]:
model_to_save = tp_model.module if hasattr(model, 'module') else tp_model
model_to_save

TensorParallelPreTrainedModel(
  (wrapped_model): TensorParallel(
    (module_shards): ModuleList(
      (0): GPTNeoXForCausalLM(
        (gpt_neox): GPTNeoXModel(
          (embed_in): TensorParallelWrapper(
            (tp_wrapped_module): Embedding(30080, 2048)
          )
          (layers): ModuleList(
            (0): GPTNeoXLayer(
              (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
              (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
              (attention): TensorParallelWrapper(
                (tp_wrapped_module): GPTNeoXAttention(
                  (rotary_emb): RotaryEmbedding()
                  (query_key_value): Linear(in_features=2048, out_features=1536, bias=True)
                  (dense): Linear(in_features=512, out_features=2048, bias=True)
                )
              )
              (mlp): TensorParallelWrapper(
                (tp_wrapped_module): GPTNeoXMLP(
                  (d

In [ ]:
import torch 

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_pretrained(output_vocab_file)

('./models/my_own_vocab_file.bin/tokenizer_config.json',
 './models/my_own_vocab_file.bin/special_tokens_map.json',
 './models/my_own_vocab_file.bin/tokenizer.json')

In [ ]:
tp_model.load_state_dict(torch.load('/home/uj-user/Yo/HiT5/HCLT/KoAlpaca/train_v1.1b/models/my_own_model_file.bin'))

<All keys matched successfully>